## Read data

In [12]:
import csv
import numpy as np

class SonarDataReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = []

    def read_data(self):
        with open(self.filepath, 'r') as file:
            reader = csv.reader(file)
            # headers = next(reader)  # 跳过表头

            for row in reader:
                pose_x = float(row[0])
                pose_y = float(row[1])
                pose_z = float(row[2])
                pose_orient_x = float(row[3])
                pose_orient_y = float(row[4])
                pose_orient_z = float(row[5])
                pose_orient_w = float(row[6])

                w_p = np.array(eval(row[7]))
                s_p = np.array(eval(row[8]))
                si_q = np.array(eval(row[9]))
                si_q_theta_Rho = np.array(eval(row[10]))
                timestep = int(row[11])
                pts_indice = np.array(eval(row[12]))

                self.data.append({
                    'pose': {
                        'position': {'x': pose_x, 'y': pose_y, 'z': pose_z},
                        'orientation': {'x': pose_orient_x, 'y': pose_orient_y, 'z': pose_orient_z, 'w': pose_orient_w}
                    },
                    'w_p': w_p,
                    's_p': s_p,
                    'si_q': si_q,
                    'si_q_theta_Rho': si_q_theta_Rho,
                    'timestep': timestep,
                    'pts_indice': pts_indice
                })

    def get_data(self):
        return self.data

filepath = "sonar_data.csv"
reader = SonarDataReader(filepath)
reader.read_data()
data = reader.get_data()

# 测试打印读取的数据
for entry in data:
    print("Pose Position: ", entry['pose']['position'])
    print("Pose Orientation: ", entry['pose']['orientation'])
    # print("w_p: ", entry['w_p'])
    # print("s_p: ", entry['s_p'])
    # print("si_q: ", entry['si_q'])
    # print("si_q_theta_d: ", entry['si_q_theta_d'])
    # print("Timestep: ", entry['timestep'])
    # print("Pts Indice: ", entry['pts_indice'])
    # print("\n")
    break

Pose Position:  {'x': -1.8282061483920884, 'y': -0.4641462448660301, 'z': -0.4141030741960442}
Pose Orientation:  {'x': -0.0776951287448076, 'y': 0.062462536984540046, 'z': -0.0776951287448076, 'w': 0.9919805267302936}


## integrate tri with ANP 

### Utility function

In [13]:
import numpy as np
import transforms3d

def quaternion_to_rotation_matrix(quaternion):
    """将四元数转换为旋转矩阵"""
    return transforms3d.quaternions.quat2mat(quaternion)

def pose_to_transform_matrix(pose):
    """将位姿转换为齐次变换矩阵"""
    position = pose['position']
    orientation = pose['orientation']
    
    # 提取平移向量
    translation = np.array([position['x'], position['y'], position['z']])
    
    # 提取四元数并转换为旋转矩阵
    quaternion = [orientation['w'], orientation['x'], orientation['y'], orientation['z']]
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    
    # 构建齐次变换矩阵
    transform_matrix = np.eye(4)
    transform_matrix[:3, :3] = rotation_matrix
    transform_matrix[:3, 3] = translation
    
    return transform_matrix


In [14]:
def compute_D(T_matrix, theta, theta_prime):
    
    R = T_matrix[:3, :3]
    t = T_matrix[:3, 3]
    
    """
    Compute the determinant D(A0; R, t) for given parameters.
    """
    def skew_symmetric_matrix(t):
        """
        Create a skew-symmetric matrix for a vector t.
        """
        return np.array([
            [0, -t[2], t[1]],
            [t[2], 0, -t[0]],
            [-t[1], t[0], 0]
        ])
    ux = np.array([1, 0, 0])
    uy = np.array([0, 1, 0])
    
    r1 = R[0, :]
    r2 = R[1, :]
        
    t_cross = skew_symmetric_matrix(t)
    
    determinant = - (r1 - np.tan(theta) * r2).T @ t_cross @ (ux - np.tan(theta_prime) * uy)
    
    return determinant

# determinant = compute_D(T_matrix, theta=theta_Rho[0], theta_prime=theta_Rho_prime[0])
# print("determiant: ", determinant)
##########################################################################


In [15]:
def get_match_pairs(si_q_theta_Rho, pts_indice, si_q_theta_Rho_prime, pts_indice_prime):
    
    # 找到共同的索引
    common_indices = np.intersect1d(pts_indice, pts_indice_prime)

    # 获取t0时刻的匹配坐标
    t0_indices = [np.where(pts_indice == idx)[0][0] for idx in common_indices]
    matched_t0 = si_q_theta_Rho[t0_indices]

    # 获取t1时刻的匹配坐标
    t1_indices = [np.where(pts_indice_prime == idx)[0][0] for idx in common_indices]
    matched_t1 = si_q_theta_Rho_prime[t1_indices]
    
    return matched_t0, matched_t1, common_indices

### Main Process

In [16]:
from anp.anp import AnPAlgorithm
anp_algorithm = AnPAlgorithm()
# t_s_cal, R_sw_cal = anp_algorithm.compute_t_R(P_SI_Noise, P_W)
from tri.tri import ANRS, GTRS

T_z_90 = np.array([[0,-1,0,0],[1,0,0,0],[0,0,1,0],[ 0,0,0,1]])
T_z_min90 = T_z_90.T
R_z_90 = T_z_90[:3, :3]

def coordinate_transform_T(T0, T1):
    # T1 = T0 @ T
    T_matrix = np.linalg.inv(T0) @ T1 
    # x-axis oriented switched to y-axis oriented
    T_matrix = T_z_90 @ T_matrix @ T_z_min90
    # get transforamtion matrix
    T_matrix = np.linalg.inv(T_matrix)
    return T_matrix

def coordinate_transform_Pose(Pose):
    return (T_z_90 @ Pose @ T_z_min90)

def coordinate_transform_pt(P):
    return (R_z_90 @ P)

def coordinate_transform(p0, p1, T0, T1):
    p0 = coordinate_transform_pt(p0)
    p1 = coordinate_transform_pt(p1)
    T_matrix = coordinate_transform_T(T0, T1)
    return p0, p1, T_matrix

# initialize
T0 = pose_to_transform_matrix(data[0]['pose'])
T1 = pose_to_transform_matrix(data[1]['pose']) # This is what we need to initialize
# T_matrix = coordinate_transform_T(T0, T1)

T0 = coordinate_transform_Pose(T0)
T1 = coordinate_transform_Pose(T1)
T_matrix = np.linalg.inv(T1) @ T0

print(T_matrix)


[[ 0.99930892  0.02943289 -0.0227023  -0.02883957]
 [-0.03002313  0.99920807 -0.02611202 -0.12115558]
 [ 0.02191577  0.02677557  0.9994012  -0.07874014]
 [ 0.          0.          0.          1.        ]]


In [17]:
theta_Rho0 = data[0]['si_q_theta_Rho']
pts_indice0 = data[0]['pts_indice']
theta_Rho1 = data[1]['si_q_theta_Rho']
pts_indice1 = data[1]['pts_indice']

s_p0_gt = data[0]['s_p']
w_p0_gt = data[0]['w_p']
s_p1_gt = data[1]['s_p']
w_p1_gt = data[1]['w_p']
print(pts_indice0)
print(pts_indice1)

[ 4  6  7 10 16 21 24 25 29 34 36 37 39 44 45 47 50 51 56 57 59 60 63 64
 68 70 71 72 74 76 81 84 88 90 95 97 99]
[ 3  4  6  7 10 16 21 24 25 29 34 37 39 44 45 47 50 51 56 57 59 60 63 64
 68 70 71 72 74 76 81 84 88 90 95 97 99]


### TRI

In [25]:
theta_Rho, theta_Rho_prime, common_indices = get_match_pairs(theta_Rho0, pts_indice0, theta_Rho1, pts_indice1)
## If we want to double check P (actually we need to calculate P)
w_P_gt = data[0]['w_p']
w_P_gt_indices = [np.where(pts_indice0 == idx)[0][0] for idx in common_indices]
w_P_gt = w_P_gt[w_P_gt_indices] 
w_P_gt = coordinate_transform_pt( w_P_gt.T ).T

In [39]:
P_dict = {}

for i in range(len(theta_Rho)):
    determinant = compute_D(T_matrix, theta=theta_Rho[i][0], theta_prime=theta_Rho_prime[i][0])
    s_P = ANRS(T_matrix, theta_Rho[i], theta_Rho_prime[i])
    w_P = ( T0 @ np.hstack([s_P, 1]) )[:3]
    key = common_indices[i]
    P_dict[key] = w_P
    # difference = w_P - coordinate_transform_pt( w_P_gt[i] )
    # print(difference, theta_Rho[i])
    # # print(w_P)
    # print(w_P_gt[i])

### ANP

In [43]:
theta_Rho2 = data[2]['si_q_theta_Rho']
pts_indice2 = data[2]['pts_indice']
q_si_x2 = np.sin(theta_Rho2.T[0]) * theta_Rho2.T[1]
q_si_y2 = np.cos(theta_Rho2.T[0]) * theta_Rho2.T[1]
q_si2 = np.vstack([q_si_x2, q_si_y2])

# Find matching pairs of q_si and 
filtered_P_w_values = []
filtered_q_si_index = []
common_indice2 = []
for i, idx in enumerate( pts_indice2 ):
    value = P_dict.get(idx)
    if value is not None:
        filtered_P_w_values.append(value[:3])
        filtered_q_si_index.append(i)
        common_indice2.append(idx)
        
q_si2 = q_si2.T[filtered_q_si_index].T
P_w2 = np.array(filtered_P_w_values).T
# print(q_si2)
# print(common_indice2)
# print(P_w2)

# Testing From dataset
# theta_Rho2 = data[2]['si_q_theta_Rho']
# pts_indice2 = data[2]['pts_indice']
# q_si_x2 = np.sin(theta_Rho2.T[0]) * theta_Rho2.T[1]
# q_si_y2 = np.cos(theta_Rho2.T[0]) * theta_Rho2.T[1]
# q_si2 = np.vstack([q_si_x2, q_si_y2])

# P_w2 = data[2]['w_p'].T
# P_w2 = coordinate_transform_pt(P_w2) 
# print(q_si2)
# print(pts_indice2)
# print(P_w2)
# pts_indice2 = data[2]['pts_indice']

In [44]:
t_s_cal, R_sw_cal = anp_algorithm.compute_t_R(q_si2, P_w2)
T2 = np.eye(4)  # 创建一个 4x4 的单位矩阵
T2[:3, :3] = R_sw_cal  # 将 R 赋值给 T 的左上 3x3 子矩阵
T2[:3, 3] = t_s_cal.flatten()  # 将 t 赋值给 T 的前 3 行第 4 列
print(np.linalg.inv(T2))

T2_gt = pose_to_transform_matrix(data[2]['pose'])
T2_gt = coordinate_transform_Pose(T2_gt)
print(T2_gt)

[[ 0.99015659  0.10717696 -0.09001711  0.45674852]
 [-0.09001749  0.98012189  0.176799   -1.54458917]
 [ 0.10717649 -0.16695556  0.98012195 -0.27229409]
 [ 0.          0.          0.          1.        ]]
[[ 0.99015653  0.10717691 -0.09001756  0.45674854]
 [-0.09001756  0.98012199  0.17679855 -1.54458916]
 [ 0.10717691 -0.16695508  0.98012199 -0.27229458]
 [ 0.          0.          0.          1.        ]]


In [45]:
T2_gt = pose_to_transform_matrix(data[3]['pose'])
T2_gt = coordinate_transform_Pose(T2_gt)
print(T2_gt)

[[ 0.99561423  0.07209013 -0.05962657  0.34147675]
 [-0.05962657  0.98009755  0.18935012 -1.36710908]
 [ 0.07209013 -0.18496434  0.98009755 -0.18355454]
 [ 0.          0.          0.          1.        ]]
